# Imports

In [ ]:
import pandas as pd 
import numpy as np

### Base com atributos boleandos porem categóricos

In [ ]:
# Base tirada de https://archive.ics.uci.edu/ml/datasets/Acute+Inflammations

# Esta base apresenta 2 colunas de classes, necessário observação extra, para isso vamos transformar em uma coluna

diagnosis = pd.read_csv('databases/diagnosis.csv',
                      header=None, 
                      names=[
                         "Temperature",
                         "nausea?",
                         "Lumbar pain?",
                         "Urine pushing?", 
                         "Micturition pains?", 
                         "Burning of urethra, itch, swelling of urethra outlet?", 
                         "decision: Inflammation of urinary bladder", 
                         "decision: Nephritis of renal pelvis origin"
                      ])

diagnosis

In [ ]:
# diagnosis.describe()

In [ ]:
# diagnosis.dtypes

## Pré-Processamento Inicial

##### Antes é necessário tratar as duas classes como uma só

In [ ]:
# Para transformar as duas colunas de classes em uma só, é feita uma comparação de condição

# A classse nova será da seguinte forma

significado_da_classe = {
    0 : "Nenhum dos dois (Inflammation of urinary bladder | Nephritis of renal pelvis origin)",
    1 : "Apenas (Inflammation of urinary bladder)",
    2 : "Apenas (Nephritis of renal pelvis origin)",
    3 : "Ambos (Inflammation of urinary bladder | Nephritis of renal pelvis origin)", 
}

def comparar_e_transformar(row):
    d_1 = len(row) - 2
    d_2 = len(row) - 1
    if row[d_1] == "no" == row[d_2]:
        return 0
    if row[d_1] == "yes" != row[d_2]:
        return 1
    if row[d_1] == "no" != row[d_2]:
        return 2
    if row[d_1] == "yes" == row[d_2]:
        return 3 

_temp = diagnosis.apply(comparar_e_transformar, axis=1)

# Deletando as duas ultimas colunas de classes para adicionar a nova

for i in range(2):
    name = diagnosis.columns[-1]
    del diagnosis[name]

diagnosis["Class"] = _temp
diagnosis

##### Converter os atributos boleanos em numeros

In [ ]:
# converter os atributos boleanos em numeros com o OrdinalEncoder

from sklearn.preprocessing import OrdinalEncoder
atributos = [
    "nausea?",
    "Lumbar pain?",
    "Urine pushing?", 
    "Micturition pains?", 
    "Burning of urethra, itch, swelling of urethra outlet?"
    ]

enc = OrdinalEncoder()
enc.fit(diagnosis[atributos])
diagnosis[atributos] = enc.transform(diagnosis[atributos])

diagnosis

In [ ]:
from sklearn.preprocessing import StandardScaler


atributos = diagnosis.columns.values.tolist()

limiar = 6 # O limiar separa o index do que é atributo para o que é a classe
atributos = diagnosis.columns[:limiar] # seleciono apenas os atributos

# Agora é possivel aplicar o StandardScaler apenas nos atributos

diagnosis[atributos] = StandardScaler().fit_transform(diagnosis[atributos])
diagnosis[atributos]

## Aprendizagem supervisionada
##### Selecionando o X e y

In [ ]:
# Selecionando o X e o y


# X = jogo_da_velha[atributos]
X = diagnosis.iloc[:, :limiar] # Atributos do Data Frame do Pandas
y = diagnosis.iloc[:, limiar:] # Classes do Data Frame do Pandas

y = np.squeeze(y) # Converter em um unico Array quando usar o dataFrame do Pandas

y_unique = np.unique(y)

mapa_cores = y.copy()

y_unique # Valores unicos de classes

##### PCA

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print('Projecting %d-dimensional data to 2D' % X.shape[1])

def plot_g(pyplot, title, colors, n_classes):
    pyplot.scatter(X_reduced[:, 0], X_reduced[:, 1], c=colors, 
             edgecolor='none', alpha=0.7, s=40,
             cmap=plt.cm.get_cmap('nipy_spectral', n_classes)
           )
    pyplot.colorbar()
    pyplot.title(title)

plt.figure(figsize=(10, 8)) # Tamanho da Figura
plot_g(plt, 'Projeção 2D do Dados em PCA', mapa_cores, len(y_unique))
plt.show()


##### Plotando os Gráficos 

In [ ]:

# plt.figure(figsize=(10*2, 7))

# plt.subplot(1, 2, 1) # row 1, col 2 index 1
# plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels, 
#             edgecolor='none', alpha=0.7, s=40,
#             cmap=plt.cm.get_cmap('nipy_spectral', len(y_unique)))
# plt.colorbar()
# plt.title('Projeção KMeans com X Original')

# plt.subplot(1, 2, 2) # row 1, col 2 index 1
# plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels_PCA, 
#             edgecolor='none', alpha=0.7, s=40,
#             cmap=plt.cm.get_cmap('nipy_spectral', len(y_unique)))
# plt.colorbar()
# plt.title('Projeção KMeans com X PCA')

# plt.show()

# Utilização do k-NN

##### Sem PCA

In [ ]:
dataset = diagnosis

i_sort = np.random.randint(len(X) - 1) # Selecionando aleatóriamente um index de teste

X_teste = X.iloc[i_sort] 
X = X.drop(i_sort)

y_teste = y.iloc[i_sort]
y = y.drop(i_sort)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X, y)

predict = knn.predict(np.array(X_teste).reshape(1,-1))

print({"Previsto:": predict, "Correto:": y_teste})

In [ ]:
dist, vizinhos = knn.kneighbors(np.array(X_teste).reshape(1,-1))
classes = []
for v in vizinhos:
    classes.append(dataset.iloc[:-1, limiar][v])
classes